In [ ]:
%matplotlib widget

#Python libraries
import os                            # find files
from matplotlib import pyplot as plt # plotting
import numpy as np                   # math library
from tqdm import tqdm           # progress bar
from collections import defaultdict  # special dictionary type
from time import time                # time module

# Ping imports
import themachinethatgoesping as ping                       # main module
import themachinethatgoesping.echosounders.simrad as simrad # simrad submodule

           
# print file names
print(ping.version())

In [ ]:
#load files
folders=[]
with open('test_data.txt','r') as ifi:
    for line in ifi:
        print("Folder:", line)
        folders.append(line)

if not len(folders):
    print("Please add test_data folders to 'test_data.txt'")
    
#find .raw files in "folders"
files = []
for folder in folders:
    print(folder)
    for r,d,f in os.walk(folder):
        for file in f:
            if file.endswith('.raw'):
                files.append(r + '/' +file)
                
files.sort()
#for f in files:
#    print(f)
    
#print()
print(len(files))

In [ ]:
# Open raw files
fm = simrad.FileSimradRaw_mapped(files)

In [ ]:
p = fm.pings()

In [ ]:
# print file info
print(fm)

In [ ]:
#%%timeit -n1 -r1
pings_all = fm.pings().get_sorted_by_time()
print("Avaliable pings:",len(pings_all))

In [ ]:
(pings_all[-1].get_timestamp() - pings_all[0].get_timestamp()) / 3600

In [ ]:
containers = pings_all.break_by_time_diff(6000)
print(len(containers))

In [ ]:
pings = containers[0]

print("Channel ids")
for cid in pings.find_channel_ids():
    print("-",cid)
    
fpings = pings("WBT 400058-15 ES333-7C_ES")
print("Avaliable pings:",len(fpings))

In [ ]:
#select pings
#pings = fpings(0,-1)
pings = fpings
#len(pings)

In [ ]:
if len(pings) > 10000:
    index = np.round(np.linspace(0,len(pings)-1,10000)).astype(int)
else:
    index = np.arange(0,len(pings)-1)
index

In [ ]:
def get_image(pmin, pmax, max_pings = 10000, max_samples = 4000):
    
    pings = fpings[:]
    
    if pmax - pmin > max_pings:
        index = np.round(np.linspace(pmin,pmax,max_pings)).astype(int)
    else:
        index = np.arange(pmin,pmax)
    index
    
    # create an empty image
    nsamples = min(4000,pings.max_number_of_samples())
    image = np.empty((len(index),nsamples),dtype=np.float32)
    image.fill(np.nan)

    # loop through pings and add sv to the image
    #for i,ping in enumerate(tqdm(pings)):
    for i,I in enumerate(tqdm(index)):
        ping = pings[I]
        sv = ping.get_sv_stacked(dB = True)
        ns = min(nsamples,len(sv))
        image[i,:ns] = sv[:ns]
        
    return image, index, nsamples
    

In [ ]:
plt.close("echo")
fig = plt.figure("echo")
fig.set_figwidth(10)
ax = fig.subplots()

image, index, nsamples = get_image(0,len(fpings)-1)
ax.imshow(image.transpose(), aspect = 'auto', vmin = -140, vmax =-40, extent=[min(index)-0.5,max(index)+0.5,nsamples+0.5,-0.5])

In [ ]:
plt.close("echo2")
fig = plt.figure("echo2")
fig.set_figwidth(10)
ax = fig.subplots()

image, index, nsamples = get_image(1070,1110,200)
ax.imshow(image.transpose(), aspect = 'auto', vmin = -140, vmax =-40, extent=[min(index)-0.5,max(index)+0.5,nsamples+0.5,-0.5])